In [1]:
%reset -f
import h5py
import time as t
import numpy as np
import scipy as sp
import scipy.io as spi
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow_probability as tfp

In [2]:
isamp = 3000
t = np.arange(0,1,0.01)
pps = 20
tsamp = pps*isamp

data_train = spi.loadmat('AD_5000_DP_TrData.mat')

u_in = data_train['u_in'][0:tsamp,:]
x_t_in = data_train['x_t_in'][0:tsamp,:]
s_in = data_train['s_in'][0:tsamp,:]

tsamp, t.shape, u_in.shape, x_t_in.shape, s_in.shape

(60000, (100,), (60000, 100), (60000, 1), (60000, 1))

In [3]:
max_u = np.max(u_in)
min_u = np.min(u_in)

u_in = (u_in-min_u)/(max_u-min_u)

max_t = np.max(x_t_in)
min_t = np.min(x_t_in)

x_t_in = (x_t_in-min_t)/(max_t-min_t)

max_s = np.max(s_in)
min_s = np.min(s_in)

s_in = (s_in-min_s)/(max_s-min_s)

In [4]:
max_s, min_s, max_u, min_u, max_t, min_t

(2.814752324700168,
 -3.0266588699038914,
 3.6565790163373184,
 -3.6177953494816775,
 0.99,
 0.0)

In [5]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Lambda, Dense

bs = tsamp

def fn(x):
    y = tf.einsum("ij, ij->i", x[0], x[1])
    y = tf.expand_dims(y, axis = 1)
    return y

tfd = tfp.distributions
tfb = tfp.bijectors

def normal_sp(params):
    return tfd.Normal(loc = params[:, 0:1], scale = 0.001+tf.math.softplus(params[:, 1:2]))    

def negloglikelihood(y_true, y_pred):
    return tf.keras.backend.sum(-y_pred.log_prob(y_true))+(sum(model.losses)/bs)

hln = 30

inputsB = Input(shape = (100,), name = 'inputsB')
hiddenB = tfp.layers.DenseFlipout(hln, activation = "relu")(inputsB)
hiddenB = tfp.layers.DenseFlipout(hln, activation = "relu")(hiddenB)
hiddenB = tfp.layers.DenseFlipout(hln, activation = "relu")(hiddenB)

inputsT = Input(shape = (1,), name = 'inputsT')
hiddenT = tfp.layers.DenseFlipout(hln, activation = "relu")(inputsT)
hiddenT = tfp.layers.DenseFlipout(hln, activation = "relu")(hiddenT)
hiddenT = tfp.layers.DenseFlipout(hln, activation = "relu")(hiddenT)

combined = Lambda(fn, output_shape = [None, 1])([hiddenB, hiddenT])
output = tfp.layers.DenseFlipout(2)(combined)

dist = tfp.layers.DistributionLambda(normal_sp)(output)
model = Model(inputs = [inputsB, inputsT], outputs = dist)    

model.summary()

/home/user/anaconda3/envs/SG_Env_TFP/lib/python3.9/site-packages/tensorflow_probability/python/layers/util.py:99: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use `layer.add_weight` method instead.
  loc = add_variable_fn(
2022-10-27 23:05:21.327955: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2022-10-27 23:05:21.328191: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-10-27 23:05:21.329097: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is opt

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inputsB (InputLayer)           [(None, 100)]        0           []                               
                                                                                                  
 inputsT (InputLayer)           [(None, 1)]          0           []                               
                                                                                                  
 dense_flipout (DenseFlipout)   (None, 30)           6030        ['inputsB[0][0]']                
                                                                                                  
 dense_flipout_3 (DenseFlipout)  (None, 30)          90          ['inputsT[0][0]']                
                                                                                              

2022-10-27 23:05:21.969633: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


In [6]:
optimizer = tf.keras.optimizers.Adam(learning_rate = 1e-3)
spe = 25
string = './model/model_s1/'+str(spe)

@tf.function
def train_step():
    with tf.GradientTape() as tape:
        loss_value = 0
        for i in range(0,spe):        
            logits = model({"inputsB":u_in, "inputsT":x_t_in}, training=True)
            loss_value = loss_value + negloglikelihood(s_in, logits)
        loss_value = loss_value*(1/spe)
    grads = tape.gradient(loss_value, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    return loss_value

epochs = 10000
loss = np.zeros(epochs)

for epoch in range(epochs):
    loss_value = train_step()
    loss[epoch] = loss_value.numpy()
    if loss[epoch] <= np.min(loss[0:epoch+1]):
        model.save_weights(string)
        last_saved_wt = epoch
    if epoch%10 == 0:
        print("Epoch %d, loss %.2f" % (epoch, loss[epoch]))

print(last_saved_wt)

Epoch 0, loss 61862.70
Epoch 10, loss 59318.86
Epoch 20, loss 51696.09
Epoch 30, loss 36797.39
Epoch 40, loss 33886.47
Epoch 50, loss 32718.49
Epoch 60, loss 25661.19
Epoch 70, loss 20453.05
Epoch 80, loss 21632.72
Epoch 90, loss 18236.32
Epoch 100, loss 16205.83
Epoch 110, loss 16280.46
Epoch 120, loss 12973.69
Epoch 130, loss 10842.17
Epoch 140, loss 6509.04
Epoch 150, loss 5651.76
Epoch 160, loss -904.15
Epoch 170, loss -2002.51
Epoch 180, loss -3941.07
Epoch 190, loss -8683.83
Epoch 200, loss -3031.87
Epoch 210, loss -4778.83
Epoch 220, loss -9710.12
Epoch 230, loss -11934.48
Epoch 240, loss -9864.29
Epoch 250, loss -12458.71
Epoch 260, loss -15856.35
Epoch 270, loss -12512.74
Epoch 280, loss -13369.27
Epoch 290, loss -18669.52
Epoch 300, loss -20352.02
Epoch 310, loss -19724.29
Epoch 320, loss -20655.73
Epoch 330, loss -19225.70
Epoch 340, loss -25109.25
Epoch 350, loss -18573.96
Epoch 360, loss -28052.98
Epoch 370, loss -21889.90
Epoch 380, loss -24944.25
Epoch 390, loss -28859.6

In [7]:
# plt.figure(figsize = [20,10])
# plt.plot(loss)
# plt.show()

In [8]:
# string = './model/model_/25'
# model.load_weights(string)

In [9]:
t = np.arange(0,1,0.01)
testdata = spi.loadmat('AD_TestData.mat')

u_in_test = testdata['u_in_test']
x_t_in_test = testdata['x_t_in_test']
s_in_test = testdata['s_in_test']

u_in_test.shape, x_t_in_test.shape, s_in_test.shape

((1000000, 100), (1000000, 1), (1000000, 1))

In [10]:
u_in_test = (u_in_test-min_u)/(max_u-min_u)
x_t_in_test = (x_t_in_test-min_t)/(max_t-min_t)
s_in_test = (s_in_test-min_s)/(max_s-min_s)

In [11]:
nsamples = 100
nps = 1
pred = np.zeros([nsamples*nps,1000000])
for i in range(0,nsamples):
    if i%5 == 0:
        print(i)
    pred[nps*i:nps*(i+1),:] = np.squeeze((model({"inputsB":u_in_test, "inputsT":x_t_in_test})).sample(nps))

pred = (pred*(max_s-min_s))+min_s 
s_in_test = (s_in_test*(max_s-min_s))+min_s

print()
print(np.mean((s_in_test-np.mean(pred, axis = 0)[..., np.newaxis])**2))
print(np.mean((s_in_test)**2))
print(np.mean((s_in_test-np.mean(pred, axis = 0)[..., np.newaxis])**2)/np.mean((s_in_test)**2))    

0
5
10
15
20
25
30
35
40
45
50
55
60
65
70
75
80
85
90
95

3.803934331100773e-05
0.28067175942433253
0.00013552964284339734


In [14]:
model.save_weights('./model/model_s1')